# Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt

from pandas.plotting import register_matplotlib_converters
from IPython.display import display
from tqdm import tqdm
# from tsa_functions import *
from tsa_tools import *  # See last cell

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)

import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

# Loading the Data

In [81]:
df_calendar = pd.read_csv('../data/m5/calendar.csv')
df_price = pd.read_csv('../data/m5/sell_prices.csv')
df_sales = pd.read_csv('../data/m5/sales_train_validation.csv')
df_sales1 = pd.read_csv('../data/m5/sales_train_evaluation.csv')
df_weights = pd.read_csv('../data/m5/weights_validation.csv')
levels1 = json.loads(open('levels1.json', 'r').read())

In [82]:
full_df = (df_sales1.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]))

In [93]:
lvl9 = full_df.sum(level=levels1['9'], axis=1)
lvl9.head()

store_id        CA_1                                                    \
state_id          CA                                                     
cat_id       HOBBIES             HOUSEHOLD               FOODS           
dept_id    HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2 FOODS_1 FOODS_2   
date                                                                     
2011-01-29       528        28         361         181     297     674   
2011-01-30       489         9         350         170     284     655   
2011-01-31       409         6         279         114     214     396   
2011-02-01       383         9         278         123     175     476   
2011-02-02       263         5         195         135     182     354   

store_id                CA_2                        ...    WI_2          \
state_id                  CA                        ...      WI           
cat_id               HOBBIES             HOUSEHOLD  ...   FOODS           
dept_id    FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1  ... FOODS_1 FOODS_2   
date                                                ...                   
2011-01-29    2268       522        16         529  ...     191     155   
2011-01-30    2198       381        16         461  ...     184     128   
2011-01-31    1398       352        16         306  ...     154     199   
2011-02-01    1607       344         6         270  ...     205     244   
2011-02-02    1496       283        13         217  ...      96     147   

store_id                WI_3                                            \
state_id                  WI                                             
cat_id               HOBBIES             HOUSEHOLD               FOODS   
dept_id    FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2 FOODS_1   
date                                                                     
2011-01-29    1269       256        22         584         148     152   
2011-01-30    1121       342        14         541         195     138   
2011-01-31    1233       228        20         420         106     127   
2011-02-01    1564       183        11         327          94      98   
2011-02-02     724        70         4         151          53      87   

store_id                    
state_id                    
cat_id                      
dept_id    FOODS_2 FOODS_3  
date                        
2011-01-29     583    2293  
2011-01-30     585    2383  
2011-01-31     575    1841  
2011-02-01     533    1965  
2011-02-02     340    1427  

[5 rows x 70 columns]

# Pre-procesing

## Pre-processing Full Dataset

## Pre-processing Lvl9

In [94]:
# No of values with less than 10 sales per day
lvl9[lvl9 < 10].count(axis=0).sum()

2403

In [95]:
df_lvl9 = (lvl9.apply(lambda x: np.where(x < 10,  np.nan, x))
            .interpolate(method='linear', axis=0)).fillna(method='bfill')

In [96]:
# To check if there are still values less than 10
df_lvl9[df_lvl9 < 10].count(axis=0).sum()

0

# Q1. Baseline Methods

In [97]:
train = df_lvl9.iloc[ :1913,:]
test = df_lvl9.iloc[ 1913:,:]

In [89]:
base_forcast = {}
h = 28
m = 7


# for bm in baseline:
naive = []
snaive = []
ses = []
hl = []
ahl = []
for x in train.columns:
    naive.append(naivef(train[x], h))
    snaive.append(snaivef(train[x], h, m))

    model1 =  ETSModel(train[x].values).fit() # SES
    model2 =  ETSModel(train[x].values, trend="add", seasonal=None).fit() # Holt's Linear
    model3 =  ETSModel(train[x], error="add", trend="add", 
                        seasonal="add", damped_trend=False, 
                        seasonal_periods=m).fit() # Additive Holt-Winter

    ses.append(model1.forecast(h))
    hl.append(model2.forecast(h))
    ahl.append(model3.forecast(h))

base_forcast['Naive'] = naive
base_forcast['Seasonal Naive'] = snaive
base_forcast['SES'] = ses
base_forcast['Holt\'s Linear'] = hl
base_forcast['Additive Holt-Winters'] = ahl


In [90]:
rmsse_res = {}

for k in base_forcast.keys():
    rmsse_list = []
    for x in range(len(train.columns)):
        rmsse_list.append(rmsse(test[train.columns[x]], 
                                base_forcast[k][x], 
                                train[train.columns[x]]))
    rmsse_res[k] = rmsse_list


In [201]:
pd.set_option('max_rows', None)
lvl9_rmsse = pd.DataFrame(rmsse_res, index=train.columns)
lvl9_rmsse_conc = (lvl9_rmsse
                   .reset_index(level=['state_id', 'cat_id'], drop=True))

lvl9_weights = (df_weights[df_weights['Level_id'] == 'Level9']
                .set_index(['Agg_Level_1', 'Agg_Level_2'])
                [['Weight']])
lvl9_weights.index.names = ['store_id', 'dept_id']

lvl9_fin = lvl9_rmsse.join(lvl9_weights)

#ncat([lvl9_rmsse_conc, lvl9_weights], axis=1).reindex(lvl9_rmsse.index, level=0)
         #).set_index(["store_id", "state_id", "cat_id", "dept_id"])
#lvl9_fin = lvl9_rmsse.merge(lvl9_weights, left_index=True, right_on=['store_id', 'dept_id'])
lvl9_fin


Naive  Seasonal Naive     SES  \
store_id dept_id     state_id cat_id                                      
CA_1     HOBBIES_1   CA       HOBBIES    1.4583          0.7619  0.8819   
         HOBBIES_2   CA       HOBBIES    1.9340          1.1429  0.8831   
         HOUSEHOLD_1 CA       HOUSEHOLD  2.1052          0.5172  1.1514   
         HOUSEHOLD_2 CA       HOUSEHOLD  2.2997          0.5228  1.2208   
         FOODS_1     CA       FOODS      0.9319          0.7312  0.9179   
         FOODS_2     CA       FOODS      2.0535          0.8269  2.0534   
         FOODS_3     CA       FOODS      1.7113          0.4944  1.0825   
CA_2     HOBBIES_1   CA       HOBBIES    1.2053          0.7055  1.1179   
         HOBBIES_2   CA       HOBBIES    1.3546          1.4893  1.3887   
         HOUSEHOLD_1 CA       HOUSEHOLD  2.0287          0.6373  1.3960   
         HOUSEHOLD_2 CA       HOUSEHOLD  1.9340          0.5920  1.4625   
         FOODS_1     CA       FOODS      1.2557          1.1311  1.3875   
         FOODS_2     CA       FOODS      4.2936          1.4906  2.3041   
         FOODS_3     CA       FOODS      2.5830          0.6046  1.4502   
CA_3     HOBBIES_1   CA       HOBBIES    0.9370          0.6647  0.6999   
         HOBBIES_2   CA       HOBBIES    1.1984          1.6617  1.5530   
         HOUSEHOLD_1 CA       HOUSEHOLD  2.6217          0.7564  1.1273   
         HOUSEHOLD_2 CA       HOUSEHOLD  2.1039          0.8808  1.4258   
         FOODS_1     CA       FOODS      1.1674          1.5874  1.3966   
         FOODS_2     CA       FOODS      1.8816          0.7316  1.7271   
         FOODS_3     CA       FOODS      1.5710          0.4353  0.8908   
CA_4     HOBBIES_1   CA       HOBBIES    1.4556          1.1008  0.9956   
         HOBBIES_2   CA       HOBBIES    2.1142          1.8563  1.8756   
         HOUSEHOLD_1 CA       HOUSEHOLD  3.4088          1.4737  1.2447   
         HOUSEHOLD_2 CA       HOUSEHOLD  1.3534          0.9567  1.0520   
         FOODS_1     CA       FOODS      1.0167          1.5664  1.3857   
         FOODS_2     CA       FOODS      1.2989          0.7998  1.1609   
         FOODS_3     CA       FOODS      1.4144          0.6385  0.9469   
TX_1     HOBBIES_1   TX       HOBBIES    1.6331          0.9169  0.9061   
         HOBBIES_2   TX       HOBBIES    1.8823          1.6475  1.4296   
         HOUSEHOLD_1 TX       HOUSEHOLD  1.8399          0.8196  1.1676   
         HOUSEHOLD_2 TX       HOUSEHOLD  1.2045          1.2523  1.0488   
         FOODS_1     TX       FOODS      1.3344          1.7189  1.5566   
         FOODS_2     TX       FOODS      1.1906          0.9520  0.8562   
         FOODS_3     TX       FOODS      1.1678          0.8416  0.9161   
TX_2     HOBBIES_1   TX       HOBBIES    0.7979          0.8792  0.8789   
         HOBBIES_2   TX       HOBBIES    0.7499          1.2548  0.9735   
         HOUSEHOLD_1 TX       HOUSEHOLD  0.8662          0.8529  0.8683   
         HOUSEHOLD_2 TX       HOUSEHOLD  1.3706          0.6559  0.9314   
         FOODS_1     TX       FOODS      1.4774          1.2816  1.2919   
         FOODS_2     TX       FOODS      0.9424          0.6834  0.7532   
         FOODS_3     TX       FOODS      0.8228          0.6226  0.7499   
TX_3     HOBBIES_1   TX       HOBBIES    1.5309          0.9025  0.9404   
         HOBBIES_2   TX       HOBBIES    1.4339          1.7212  1.1186   
         HOUSEHOLD_1 TX       HOUSEHOLD  1.0738          1.0794  1.0248   
         HOUSEHOLD_2 TX       HOUSEHOLD  0.8605          1.3137  1.0128   
         FOODS_1     TX       FOODS      2.7332          2.2206  2.1542   
         FOODS_2     TX       FOODS      1.3682          1.4195  1.4254   
         FOODS_3     TX       FOODS      0.9672          1.1046  1.0692   
WI_1     HOBBIES_1   WI       HOBBIES    1.3685          0.5084  0.7533   
         HOBBIES_2   WI       HOBBIES    1.1278          1.6404  1.2088   
         HOUSEHOLD_1 WI       HOUSEHOLD  1.7201          0.6488  1.

In [205]:
lvl9_wrmsse = {'Naive': sum(lvl9_fin['Naive'] * lvl9_fin['Weight']),
               'S. Naive': sum(lvl9_fin['Seasonal Naive'] * 
                                lvl9_fin['Weight']),
               'SES': sum(lvl9_fin['SES'] * lvl9_fin['Weight']),
               'Holt\'s Linear': sum(lvl9_fin['Holt\'s Linear'] * 
                                      lvl9_fin['Weight']),
               'Additive Holt-Winters': sum(lvl9_fin['Additive Holt-Winters'] 
                                             * lvl9_fin['Weight'])}
pd.DataFrame.from_dict(lvl9_wrmsse, orient='index', columns=['WRMSSE'])

,WRMSSE
Naive,1.6286
S. Naive,0.9300
SES,1.2075
Holt's Linear,1.2367
Additive Holt-Winters,0.8549


# Part 4: Middle-Out Method

In [33]:
import json
levels = json.loads(open('levels.json', 'r').read())
levels


{'1': None,
 '2': 'state_id',
 '3': 'store_id',
 '4': 'cat_id',
 '5': 'dept_id',
 '6': ['state_id', 'cat_id'],
 '7': ['state_id', 'dept_id'],
 '8': ['store_id', 'cat_id'],
 '9': ['store_id', 'dept_id'],
 '10': 'item_id',
 '11': ['state_id', 'item_id'],
 '12': ['store_id', 'item_id']}

In [21]:
sample = pd.DataFrame(base_forcast['Naive'], index=train.columns).T

In [36]:
full_df.sum(level=[levels[str(2)]], axis=1).shape

(1941, 3)

## Bottom-Up

In [ ]:
def compute_bottom_up(df_orig, df_pred, lvl_pred):
    """Returns a dictionary of dataframes of RMSSEs for each time series"""

In [68]:
low = full_df.sum(level=["state_id", "store_id", "cat_id"], axis=1)
print(low.shape, low.columns[0])

(1941, 30) ('CA', 'CA_1', 'HOBBIES')


In [78]:
low = full_df.sum(level=["state_id", "cat_id"], axis=1)
print(low.shape, low.columns[0])

(1941, 9) ('CA', 'HOBBIES')


In [79]:
# full_df.sum(level=["state_id", "cat_id"], axis=1)

In [54]:
for x in list(sorted(range(1, 9), reverse=True)):
    try:
        print(full_df.sum(level=[levels[str(x)]], axis=1).shape)
    except:
        print(full_df.sum(level=levels[str(x)], axis=1).shape)

(1941, 30)
(1941, 21)
(1941, 9)
(1941, 7)
(1941, 3)
(1941, 10)
(1941, 3)
(1941,)


In [38]:
df_pred = sample
df_pred

store_id      CA_1                                                            \
dept_id  HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2 FOODS_1 FOODS_2 FOODS_3   
0            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
1            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
2            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
3            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
4            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
5            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
6            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
7            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
8            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
9            641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
10           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
11           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
12           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
13           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
14           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
15           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
16           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
17           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
18           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
19           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
20           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
21           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
22           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
23           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
24           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
25           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
26           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   
27           641.0      73.0      1022.0       324.0   329.0   734.0  2990.0   

store_id      CA_2                        ...    WI_2                  \
dept_id  HOBBIES_1 HOBBIES_2 HOUSEHOLD_1  ... FOODS_1 FOODS_2 FOODS_3   
0            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
1            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
2            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
3            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
4            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
5            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
6            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
7            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
8            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
9            447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
10           447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
11           447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
12           447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
13           447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
14           447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
15           447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
16           447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
17           447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
18           447.0      60.0      1131.0  ...   347.0   932.0  2380.0   
19           447.0      60.0      1131.0  ...   347.0   932.0  238

## Top-Down Approach

In [ ]:
def compute_topdown(df_full, df_pred, level, approach='AHP'):
    """Returns"""

    if approach == 'AHP':

    elif approach == 'PHA':
    
    else:
        

In [45]:

for col in df_pivot.columns.tolist()[0:-1]:
    state_forecasted[col] = np.array(df_totalf) * np.array(df_stateprop[col])

df_statef = pd.DataFrame(state_forecasted, index=df_test.index)

<function dict.values>

In [57]:
proportions = {}
next_level_forecasted = {}
lvl = full_df.sum(level=levels[str(10)], axis=1)
for col in lvl.columns.tolist():
    proportions[col] = sum(lvl[col]/lvl.sum(axis=1)) * (1/len(lvl))

next_level_forecasted = np.array(sample) * np.array(df_stateprop[col])

NameError: name 'df_stateprop' is not defined

## Compute for the WRMSSE per level

In [ ]:
def compute_wrmsse(df_rmsse):